In [4]:
import pandas as pd
from collections import Counter
import sys
import os
import ast
import json
import pandas as pd
import numpy as np
import ast
import joblib
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
import mlflow
import mlflow.sklearn
from scipy.stats import loguniform

from new_or_used import build_dataset
pd.set_option('display.max_columns', None)

mlflow.set_tracking_uri("file:../Experiments") 
mlflow.set_experiment("Neural_Network_Experiment")


<Experiment: artifact_location=('file:///Users/sebastian/Library/Mobile '
 'Documents/com~apple~CloudDocs/Meli/CodeExercise DS - Items Nuevos_Usados '
 '(1)/MercadoLibre_Test/Notebooks/../Experiments/401457704096855387'), creation_time=1747891602238, experiment_id='401457704096855387', last_update_time=1747891602238, lifecycle_stage='active', name='Neural_Network_Experiment', tags={}>

In [5]:
# Cargar los datasets preprocesados
X_train = pd.read_csv("../Data/Gold/X_train_gold.csv")
X_test = pd.read_csv("../Data/Gold/X_test_gold.csv")
y_train = pd.read_csv("../Data/Gold/y_train_gold.csv")
y_test = pd.read_csv("../Data/Gold/y_test_gold.csv")

y_test.rename(columns={'0': 'condition'}, inplace=True)

y_train_final = y_train["condition"].map({"used": 0, "new": 1})
y_test_final = y_test["condition"].map({"used": 0, "new": 1})


print("Datasets cargados correctamente.")

Datasets cargados correctamente.


In [6]:
import mlflow
import mlflow.sklearn
from sklearn.neural_network import MLPClassifier


# === Agrupar ciudades top
top_cities = X_train['seller_address.city.name'].value_counts().nlargest(20).index
X_train['city_grouped'] = X_train['seller_address.city.name'].apply(lambda x: x if x in top_cities else 'other')
X_test['city_grouped'] = X_test['seller_address.city.name'].apply(lambda x: x if x in top_cities else 'other')

# === Eliminar columnas de alta cardinalidad
X_train_lr = X_train.drop(columns=['seller_id', 'seller_address.city.name'])
X_test_lr = X_test.drop(columns=['seller_id', 'seller_address.city.name'])

# === Columnas categóricas y numéricas
cat_cols = X_train_lr.select_dtypes(include='object').columns.tolist()
num_cols = X_train_lr.select_dtypes(include=['int64', 'float64']).columns.tolist()

# === Pipeline
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
    ("num", StandardScaler(), num_cols)
])

pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", MLPClassifier(hidden_layer_sizes=(64,), max_iter=200, random_state=42))
])

with mlflow.start_run(run_name="NeuralNet"):
    pipeline.fit(X_train, y_train_final)
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test_final, y_pred)
    auc = roc_auc_score(y_test_final, y_prob)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("roc_auc", auc)
    mlflow.sklearn.log_model(pipeline, "NeuralNet")

    print(f"NeuralNet => Accuracy: {acc:.4f} | ROC AUC: {auc:.4f}")

/opt/anaconda3/envs/sumz/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
python(53083) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2025/05/22 12:51:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


NeuralNet => Accuracy: 0.8625 | ROC AUC: 0.9358
